# Imports

In [187]:
# imports
# Selenium Wire
from seleniumwire import webdriver
from seleniumwire.utils import decode

# Selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait

# Else
from getpass import getpass
import time

# Classes and functions

In [188]:
# dictionary of Accounts
accounts = {}

# Post class
class Post:
    def __init__(self, likes, date_of_pub):
        self.likes = likes
        self.date_of_pub = date_of_pub

# Account class
class Account:
    def __init__(self, followers, link):
        self.posts = []  # A list to store Post objects
        self.followers = followers
        self.link = link
        
    def append_post(self, post):
        if isinstance(post, Post):
            self.posts.append(post)
        else:
            raise ValueError("Only Post objects can be added to 'posts' attribute.")

## Set up for scraping

In [189]:
# setting up webdriver
driver = webdriver.Chrome()


# open the webpage
driver.get("https://www.instagram.com")

### Logging in

In [190]:
# target username and password
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

# enter username and password
username.clear()
username.send_keys(input("Enter you username: "))
password.clear()
password.send_keys(getpass("Enter your password: "))

# target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

# We are logged in! test_username: ninamuray3 test_password: wexna4-suxkeb-gyFwov

Enter you username: ninamuray3
Enter your password: ········


### NOT NOW handling

In [191]:
# hadle NOT NOW. Comment out one of them if not needed or gives TimeOut Exception
# 1) Save details
not_now = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[contains(text(), "Not now")]'))).click()

# 2) Notifications
not_now2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]'))).click()

### Search hashtags

In [192]:
keyword = input("Enter the hashtag without '#': ")
driver.get("https://www.instagram.com/explore/tags/" + keyword + "/")
 
# Wait for 5 seconds
time.sleep(5)

Enter the hashtag without '#': ramen


### Start Scarping

In [193]:
# find and click on the first post to activate the "Next" button
first_post = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/p/')]"))).click()

In [194]:
action = ActionChains(driver)
start_time = time.time()
for i in range(70):
    
    # hover over username
    try:
        element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div._aasi > div > header > div._aaqy._aaqz > div._aar0._ad95._aar1 > div.x78zum5 > div > div > span > div > div > a')))
    except Exception as e:
        next_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Next"]'))).click()
        continue
        
    action.move_to_element(element).perform()

    # catch response from https://www.instagram.com/api/v1/users/{user_id}/info
    request = driver.wait_for_request('https://www.instagram.com/api/v1/users/', 10)
    time.sleep(2)
    response = request.response
    response_decoded = json.loads(decode(response.body, response.headers.get('Content-Encoding', 'identity')))
    print(json.dumps(response_decoded, sort_keys = True, indent = 4))

    # record the account and/or post if the account is buisness
    if response_decoded["user"]["is_business"]:
        username = response_decoded["user"]["username"]
        link = element.get_attribute('href')
        followers = response_decoded["user"]["follower_count"]
        date_of_pub = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div._ae5u._ae5v._ae5w > div > div > a > span > time'))).get_attribute('datetime')
        
        # likes request handle
        likes_request = driver.wait_for_request('/api/v1/media/.*?/info', 10)
        time.sleep(2)
        likes_response = likes_request.response
        likes_response_decoded = json.loads(decode(likes_response.body, likes_response.headers.get('Content-Encoding', 'identity')))
        likes = likes_response_decoded["items"][0]["like_count"]
        
        if username in accounts:
            accounts[username].append_post(Post(likes, date_of_pub))
        else:
            profile = Account(followers, link)
            profile.append_post(Post(likes, date_of_pub))
            accounts[username] = profile
            
    # clear previously captured requests
    del driver.requests
    
    print(accounts)
    print(i)
    print()
    print()
    
    # click "next" button
    next_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Next"]'))).click()

end_time = time.time()
duration = end_time - start_time
print(f"Total duration of the loop: {duration:.2f} seconds")
print(f"Number of business accounts: {len(accounts)}")

{
    "status": "ok",
    "user": {
        "account_badges": [],
        "account_category": "",
        "account_type": 3,
        "address_street": "",
        "ads_incentive_expiration_date": null,
        "ads_page_id": null,
        "ads_page_name": null,
        "auto_expand_chaining": null,
        "bio_links": [],
        "biography": "\u25ab\ufe0f\u30e9\u30fc\u30e1\u30f3\u30a2\u30ab\u30a6\u30f3\u30c8\ud83c\udf5c\n \u4e3b\u306b#\u5ca1\u5c71\u30e9\u30fc\u30e1\u30f3 \n\u4eba\u306b\u3088\u3063\u3066\u597d\u307f\u306f\u9055\u3046\u3051\u3069\n\u5168\u3066\u306e\u30e9\u30fc\u30e1\u30f3\u7f8e\u5473\u3057\u3044\uff01\n116\u5e97\u8217\u2728\u5e74\u9593100\u676f\u4ee5\u4e0a\u9054\u6210\u2728",
        "biography_with_entities": {
            "entities": [
                {
                    "hashtag": {
                        "id": "17843403685055752",
                        "name": "\u5ca1\u5c71\u30e9\u30fc\u30e1\u30f3"
                    }
                }
            ],
      

{
    "status": "ok",
    "user": {
        "account_badges": [],
        "account_category": "",
        "account_type": 2,
        "address_street": "",
        "ads_incentive_expiration_date": null,
        "ads_page_id": "904441063221772",
        "ads_page_name": "Hiroaki Takayama",
        "auto_expand_chaining": null,
        "bio_links": [],
        "biography": "\u3089\u30fc\u3081\u3093\ud83c\udf5c\n\u30fb\n\u30d5\u30a9\u30ed\u30fc\u3001\u30b3\u30e1\u30f3\u30c8\u5927\u6b53\u8fce\u3067\u3059\uff01",
        "biography_with_entities": {
            "entities": [],
            "raw_text": "\u3089\u30fc\u3081\u3093\ud83c\udf5c\n\u30fb\n\u30d5\u30a9\u30ed\u30fc\u3001\u30b3\u30e1\u30f3\u30c8\u5927\u6b53\u8fce\u3067\u3059\uff01"
        },
        "birthday_today_visibility_for_viewer": "NOT_VISIBLE",
        "broadcast_chat_preference_status": {
            "json_response": "{\"status\":\"ok\",\"status_code\":\"200\",\"is_broadcast_chat_creator\":true,\"notification_setting_type\":2

{'ramaru1127': <__main__.Account object at 0x7f90e0f44df0>}
2


{
    "status": "ok",
    "user": {
        "account_badges": [],
        "account_category": "",
        "account_type": 3,
        "address_street": "",
        "ads_incentive_expiration_date": null,
        "ads_page_id": "106629618788631",
        "ads_page_name": "ayac.418",
        "auto_expand_chaining": null,
        "bio_links": [
            {
                "link_id": "17988968941577000",
                "link_type": "external",
                "lynx_url": "https://l.instagram.com/?u=https%3A%2F%2Flit.link%2Fayac418&e=AT2sS1mEe14YNs1zGhZ24z0ONy8J4NJpMRptSNyMAMuuLggr7NgfLJIvgaQyhdiyfbS5_dljyLNyRIJ7eI7LHHAVP4pYHpx6n93-ho1E_eC57BaD8RIsAeE",
                "open_external_url_with_in_app_browser": true,
                "title": "\u30ea\u30f3\u30af",
                "url": "https://lit.link/ayac418"
            }
        ],
        "biography": "\\\\  \u898b\u308b\u3068\u98f2\u307f\u305f\u304f\u306a\u3063\u3061\u308

{
    "status": "ok",
    "user": {
        "account_badges": [],
        "account_category": "",
        "account_type": 3,
        "address_street": "",
        "ads_incentive_expiration_date": null,
        "ads_page_id": null,
        "ads_page_name": null,
        "auto_expand_chaining": null,
        "bio_links": [],
        "biography": "\u305f\u305f\u3099\u306e\u30e9\u30fc\u30e1\u30f3\u597d\u304d\n\u57fa\u672c #\u91d1\u753a\u88fd\u9eba #\u624b\u6253\u5f0f\u8d85\u591a\u52a0\u6c34\u9eba\u306e\u306e\u304f\u3089 #\u967d\u306f\u307e\u305f\u306e\u307c\u308b #\u4e2d\u83ef\u854e\u9ea6\u3068\u307f\u7530   \u5e38\u78d0\u7dda\u6cbf\u7dda\u4e2d\u5fc3\u306b\u9eba\u6d3b\u4e2d\ud83c\udf5c\n\u305f\u307e\u306b #\u795e\u793e\u5de1\u308a \u26e9\u3068#\u30b5\u30c3\u30ab\u30fc\u65e5\u672c\u4ee3\u8868 \u89b3\u6226\u26bd\ufe0f \n#\u30df\u30cb\u30d4\u30f3 \u63a8\u3057 #\u5c3e\u5d0e\u8c4a \u4fe1\u8005",
        "biography_with_entities": {
            "entities": [
                {
                   

{
    "status": "ok",
    "user": {
        "account_badges": [],
        "account_category": "",
        "account_type": 1,
        "ads_incentive_expiration_date": null,
        "ads_page_id": null,
        "ads_page_name": null,
        "auto_expand_chaining": null,
        "bio_links": [],
        "biography": "\u708e\u306e\u884c\u5217\uff7d\uff84\uff6f\uff8a\uff9f-\ud83d\udc7a\ud83e\uded1\n\ud83c\udf5c\ud83c\udf65\ud83d\udc5f\ud83d\udc40\u2728",
        "biography_with_entities": {
            "entities": [],
            "raw_text": "\u708e\u306e\u884c\u5217\uff7d\uff84\uff6f\uff8a\uff9f-\ud83d\udc7a\ud83e\uded1\n\ud83c\udf5c\ud83c\udf65\ud83d\udc5f\ud83d\udc40\u2728"
        },
        "birthday_today_visibility_for_viewer": "NOT_VISIBLE",
        "can_add_fb_group_link_on_profile": false,
        "can_hide_category": true,
        "can_use_affiliate_partnership_messaging_as_brand": false,
        "can_use_affiliate_partnership_messaging_as_creator": false,
        "can_use_brand

...

{
    "status": "ok",
    "user": {
        "account_badges": [],
        "account_category": "",
        "account_type": 1,
        "ads_incentive_expiration_date": null,
        "ads_page_id": "469033376822679",
        "ads_page_name": "Sch. Punya Cerita",
        "auto_expand_chaining": null,
        "bio_links": [
            {
                "link_id": "17969856862831890",
                "link_type": "external",
                "lynx_url": "https://l.instagram.com/?u=https%3A%2F%2Fyoutu.be%2FHOX1mL-XX6Q&e=AT2EdZ1F2XdEu7DQ1E-M6RrsrVVPEk77ZJVyfpan4zuUsQte7unItFeWtjzanfS3bw6DohxlJ0SrnE3PN5W1G7vlEw3fzFu0yT5j9gT8xHYGUDxH0dXE3ms",
                "open_external_url_with_in_app_browser": true,
                "title": "Sch. Video",
                "url": "https://youtu.be/HOX1mL-XX6Q"
            }
        ],
        "biography": "#SchPunyaCerita Est. 2014 hola everyone \ud83d\udc4b\nWelcome to my food journey gallery\nPart of @schpunyatoko @junz_kitchen @basecampersid\nYoutube: Sch. 

{
    "status": "ok",
    "user": {
        "account_badges": [],
        "account_category": "",
        "account_type": 3,
        "address_street": "",
        "ads_incentive_expiration_date": null,
        "ads_page_id": null,
        "ads_page_name": null,
        "auto_expand_chaining": null,
        "bio_links": [],
        "biography": "Creole with a passion for all things food and bbq. Jack of all trades\ud83c\udfb0",
        "biography_with_entities": {
            "entities": [],
            "raw_text": "Creole with a passion for all things food and bbq. Jack of all trades\ud83c\udfb0"
        },
        "birthday_today_visibility_for_viewer": "NOT_VISIBLE",
        "business_contact_method": "UNKNOWN",
        "can_add_fb_group_link_on_profile": false,
        "can_hide_category": true,
        "can_hide_public_contacts": true,
        "can_use_affiliate_partnership_messaging_as_brand": false,
        "can_use_affiliate_partnership_messaging_as_creator": false,
        "can

{
    "status": "ok",
    "user": {
        "account_badges": [],
        "account_category": "",
        "account_type": 2,
        "address_street": "515 abbott street",
        "ads_incentive_expiration_date": null,
        "ads_page_id": "101423917939337",
        "ads_page_name": "TaishokenCanada",
        "auto_expand_chaining": null,
        "bio_links": [
            {
                "link_id": "17964412639899918",
                "link_type": "external",
                "lynx_url": "https://l.instagram.com/?u=https%3A%2F%2Falmenu.io%2Fm%2FbMsqiZd5&e=AT22n-gblW9G2zSegD3LCXqBc-Xs0OqJy1s_9ZFx41m2CQ6JOyxBy0pRnwYQcDXJ_2zYltX31gB7put8nW9Oz2bSs0Zd9K60gnh3boy9AB8AYNmie3jKg9A",
                "open_external_url_with_in_app_browser": true,
                "title": "",
                "url": "https://almenu.io/m/bMsqiZd5"
            }
        ],
        "biography": "\ud83d\udef5 Ubereats , Skipthedishes, DoorDash\n\u260e\ufe0f 778-737-3805\n\ud83c\udf5c Menu \u2193\u2193\u2193\u2193\

{'ramaru1127': <__main__.Account object at 0x7f90e0f44df0>, 'taqueriaelsancho956': <__main__.Account object at 0x7f90e09488e0>, 'ranguatemala': <__main__.Account object at 0x7f90e018e910>, 'kin.sushi1': <__main__.Account object at 0x7f90e092ee80>, 'yokostreet.us': <__main__.Account object at 0x7f90e01a3d90>, 'ramensanchi': <__main__.Account object at 0x7f90df3b82b0>, 'shinyaramenhouse': <__main__.Account object at 0x7f90f54cb040>, 'tenko.udon': <__main__.Account object at 0x7f90e1859a90>, 'crunchycheese_mx': <__main__.Account object at 0x7f90e2043460>, '203exotics': <__main__.Account object at 0x7f90e0245d00>, 'gofreeforfood_': <__main__.Account object at 0x7f90dd8e38e0>, 'firenoodzhouston': <__main__.Account object at 0x7f90e185fdf0>, 'izakayakin': <__main__.Account object at 0x7f90e00bdd90>, 'deigoramen': <__main__.Account object at 0x7f90e1cfde50>, 'noodsbar': <__main__.Account object at 0x7f90df3c2310>, 'gungho.bolingbrook': <__main__.Account object at 0x7f90de2afc40>, 'masaki.gdl'

{
    "status": "ok",
    "user": {
        "account_badges": [],
        "account_category": "",
        "account_type": 1,
        "ads_incentive_expiration_date": null,
        "ads_page_id": null,
        "ads_page_name": null,
        "auto_expand_chaining": null,
        "bio_links": [],
        "biography": "\u30eb\u30fc\u30eb \uff1a\u3010\u540c\u3058\u30e9\u30fc\u30e1\u30f3\u306f\u8f09\u305b\u306a\u3044\u3011\n\u898b\u305f\u76ee\u3001\u30b9\u30fc\u30d7\u3001\u9eba\u3001\u30c8\u30c3\u30d4\u30f3\u30b0\u30920\u301c5\u3067\u70b9\u6570\u3064\u3051\u3001\n\u5e73\u5747\u304b\u3089\u3001\u7dcf\u5408\u500b\u4eba\u8a55\u4fa1\u3092\u8f9b\u53e3\u3081\u3067\u4ed8\u3051\u308b\n\u5168\u56fd(\u305f\u307e\u306b\u6d77\u5916)\u98df\u3079\u56de\u3063\u3066\u307e\u3059\u304c\u3001\u95a2\u6771\u591a\u3081\ud83c\udf5c\n\u203b\u6295\u7a3f\u65e5\u306f\u98df\u3079\u305f\u65e5\u3067\u306f\u3042\u308a\u307e\u305b\u3093\uff01\n\u2193\u3061\u3083\u3093\u3068\u3057\u305f\u30ab\u30e1\u30e9\n@sukekiyo00000",
 